# Input data for tests

As we saw in the last section, when using parametrisation it's often useful to split your test function into two parts:
1. The data to be tested
2. The code to do the test

This is because we had a situation where we had one test function and multiple examples to test. The opposite situation also happens where we have multiple test functions, all of which want the same input data.

The name that pytest uses for "data which is provided to test functions" is *fixture* since it *fixes* a set of data against which to test.

We'll start with the example of the `add_arrays` function to explain the syntax but soon we'll need to use a example which demonstates the benefits more.

To make things clearer, we'll trim down the test file back to the basics. Just one test for `add_arrays` and one for `subtract_arrays`:

In [1]:
%%writefile arrays.py

"""
This module contains functions for manipulating and combining Python lists.
"""

def add_arrays(x, y):
    """
    This function adds together each element of the two passed lists.

    Args:
        x (list): The first list to add
        y (list): The second list to add

    Returns:
        list: the pairwise sums of ``x`` and ``y``.

    Examples:
        >>> add_arrays([1, 4, 5], [4, 3, 5])
        [5, 7, 10]
    """

    if len(x) != len(y):
        raise ValueError("Both arrays must have the same length.")

    z = []
    for x_, y_ in zip(x, y):
        z.append(x_ + y_)

    return z

def subtract_arrays(x, y):
    """
    This function subtracts from each other each element of the two passed lists.

    Args:
        x (list): The first list
        y (list): The second list

    Returns:
        list: the pairwise difference of ``x`` and ``y``.

    Examples:
        >>> subtract_arrays([1, 4, 5], [4, 3, 5])
        [-3, 1, 0]
    """

    if len(x) != len(y):
        raise ValueError("Both arrays must have the same length.")

    z = []
    for x_, y_ in zip(x, y):
        z.append(x_ - y_)

    return z

Overwriting arrays.py


In [2]:
%%writefile test_arrays.py

from arrays import add_arrays, subtract_arrays

def test_add_arrays():
    a = [1, 2, 3]
    b = [4, 5, 6]
    expect = [5, 7, 9]
    
    output = add_arrays(a, b)
    
    assert output == expect

def test_subtract_arrays():
    a = [1, 2, 3]
    b = [4, 5, 6]
    expect = [-3, -3, -3]
    
    output = subtract_arrays(a, b)
    
    assert output == expect

Overwriting test_arrays.py


Both of these tests use the same values for `a` and `b`. In this example it's not really a big problem that we're repeating ourselves but in a more complex test suite the data we're testing over may be very complicated or slow to create.

To avoid repeating ourselves, we will create a fixture which will provide the data to our tests. To create our fixture we define a function which is decorated with the `pytest.fixture` decorator. Apart from that, all the function needs do is return the data we want to provide to our tests, in this case, the two input lists:

```python
import pytest

@pytest.fixture
def pair_of_lists():
    return [1, 2, 3], [4, 5, 6]
```

To make the test functions make use of the fixture, we use the name of the fixture (`pair_of_lists`) as a parameter of the test function, similar to how we did with parametrisation:

```python
def test_add_arrays(pair_of_lists):
    ...
```

The data is now available inside the function using that name and we can use it however we wish:

```python
def test_add_arrays(pair_of_lists):
    a, b = pair_of_lists
    ...
```

This isn't how functions and arguments usually work in Python. pytest is doing something magic here and is matching up the names of things which it knows are fixtures (due to the decorator) with the names of parameters to test functions, automatically running the fixture and passing in the data.

Note that `pair_of_lists` here is not a test function. It does not contain any `assert`s and will not appear in the `pytest` output.

Putting it all together, we end up with:

In [3]:
%%writefile test_arrays.py

import pytest

from arrays import add_arrays, subtract_arrays

@pytest.fixture
def pair_of_lists():
    return [1, 2, 3], [4, 5, 6]

def test_add_arrays(pair_of_lists):
    a, b = pair_of_lists
    expect = [5, 7, 9]
    
    output = add_arrays(a, b)
    
    assert output == expect

def test_subtract_arrays(pair_of_lists):
    a, b = pair_of_lists
    expect = [-3, -3, -3]
    
    output = subtract_arrays(a, b)
    
    assert output == expect

Overwriting test_arrays.py


When we run the test suite, pytest will automatically run the `pair_of_lists` function for each test and pass in the result.

In [4]:
!COLUMNS=60 venv/bin/pytest -v

=================== test session starts ====================
platform linux -- Python 3.7.3, pytest-5.2.2, py-1.8.0, pluggy-0.13.0 -- /home/matt/projects/software_engineering_best_practices/venv/bin/python3
cachedir: .pytest_cache
rootdir: /home/matt/projects/software_engineering_best_practices
plugins: nbval-0.9.3
collected 2 items                                          

test_arrays.py::test_add_arrays PASSED               [ 50%]
test_arrays.py::test_subtract_arrays PASSED          [100%]

==================== 2 passed in 0.03s =====================


## A different example

It might be hard to see the benefit of fixtures with this rather contrived example so lets take a look at a more sensible one where using a fixture makes sense.

Make a new file called `books.py` which contains the following:

In [5]:
%%writefile books.py

def word_count(text, word=''):
    """
    Count the number of occurences of ``word`` in a string.
    If ``word`` is not set, count all words.
    """
    if word:
        count = 0
        for text_word in text.split():
            if text_word == word:
                count += 1
        return count
    else:
        return len(text.split())

Writing books.py


To test this function we want a corpus of text to test it on. For the purposes of this example and to simulate a complex data input, we will download the contents of a long book from Project Gutenberg. Our test function uses [`urllib.request`](https://docs.python.org/3/library/urllib.request.html) to download the text, converts it to a string and passes that to the `word_count` function.

At first we will simply check that the word "hat" appears 33 times in the book:

In [6]:
%%writefile test_books.py

import urllib.request

from books import word_count

def test_word_counts():
    url = "https://www.gutenberg.org/files/2600/2600-0.txt"
    book_text = urllib.request.urlopen(url).read().decode('utf-8')
    assert word_count(book_text, "hat") == 33

Writing test_books.py


In [7]:
!COLUMNS=60 venv/bin/pytest -v test_books.py

=================== test session starts ====================
platform linux -- Python 3.7.3, pytest-5.2.2, py-1.8.0, pluggy-0.13.0 -- /home/matt/projects/software_engineering_best_practices/venv/bin/python3
cachedir: .pytest_cache
rootdir: /home/matt/projects/software_engineering_best_practices
plugins: nbval-0.9.3
collected 1 item                                           

test_books.py::test_word_counts PASSED               [100%]

==================== 1 passed in 2.32s =====================


The test has passed and it took about two seconds. This is because it's taking some time to download the file from the internet. For this example we *want* it to take some time as it helps demonstrate the point. In reality you will come across test data inputs which take some time (more than a few milliseconds) to create.

This crete a tension between wanting to have a large test suite which covers your code from lots of different angles and being able to run it very quickly and easily. An ideal test suite will run as quickly as possible as it will encourage you to run it more often. It's a good idea to have at least a subset of your tsts which run through in some number of seconds rather than hours.

Two seconds is not bad for this test but if we want to test against multiple examples, it could get slow. Let's parametrise the test to add in a bunch more inputs:

In [8]:
%%writefile test_books.py

import urllib.request

import pytest

from books import word_count

@pytest.mark.parametrize('word, count',  [
    ('hat', 33),
    ('freedom', 71),
    ('electricity', 1),
    ('testing', 3),
    ('Prince', 1498),
    ('internet', 0),
    ('Russia', 71),
    ('Pierre', 1260),
    (None, 566311),
])
def test_word_counts(word, count):
    url = "https://www.gutenberg.org/files/2600/2600-0.txt"
    book_text = urllib.request.urlopen(url).read().decode('utf-8')
    assert word_count(book_text, word) == count

Overwriting test_books.py


In [9]:
!COLUMNS=60 venv/bin/pytest -v test_books.py

=================== test session starts ====================
platform linux -- Python 3.7.3, pytest-5.2.2, py-1.8.0, pluggy-0.13.0 -- /home/matt/projects/software_engineering_best_practices/venv/bin/python3
cachedir: .pytest_cache
rootdir: /home/matt/projects/software_engineering_best_practices
plugins: nbval-0.9.3
collected 9 items                                          

test_books.py::test_word_counts[hat-33] PASSED       [ 11%]
test_books.py::test_word_counts[freedom-71] PASSED   [ 22%]
test_books.py::test_word_counts[electricity-1] PASSED [ 33%]
test_books.py::test_word_counts[testing-3] PASSED    [ 44%]
test_books.py::test_word_counts[Prince-1498] PASSED  [ 55%]
test_books.py::test_word_counts[internet-0] PASSED   [ 66%]
test_books.py::test_word_counts[Russia-71] PASSED    [ 77%]
test_books.py::test_word_counts[Pierre-1260] PASSED  [ 88%]
test_books.py::test_word_counts[None-566311] PASSED  [100%]

==================== 9 passed in 24.30s ====================


You see here that it took about nine times as long. This is because the file is downloaded afresh for every test example where really, it only *needs* to be downloaded once.

Let's move the slow setup into a fixture and give that as a parameter of the test function:

In [10]:
%%writefile test_books.py

import urllib.request

import pytest

from books import word_count

@pytest.fixture()
def long_book():
    url = "https://www.gutenberg.org/files/2600/2600-0.txt"
    book_text = urllib.request.urlopen(url).read().decode('utf-8')
    return book_text

@pytest.mark.parametrize('word, count',  [
    ('hat', 33),
    ('freedom', 71),
    ('electricity', 1),
    ('testing', 3),
    ('Prince', 1498),
    ('internet', 0),
    ('Russia', 71),
    ('Pierre', 1260),
    (None, 566311),
])
def test_word_counts(long_book, word, count):
    assert word_count(long_book, word) == count

Overwriting test_books.py


In [11]:
!COLUMNS=60 venv/bin/pytest -v test_books.py

=================== test session starts ====================
platform linux -- Python 3.7.3, pytest-5.2.2, py-1.8.0, pluggy-0.13.0 -- /home/matt/projects/software_engineering_best_practices/venv/bin/python3
cachedir: .pytest_cache
rootdir: /home/matt/projects/software_engineering_best_practices
plugins: nbval-0.9.3
collected 9 items                                          

test_books.py::test_word_counts[hat-33] PASSED       [ 11%]
test_books.py::test_word_counts[freedom-71] PASSED   [ 22%]
test_books.py::test_word_counts[electricity-1] PASSED [ 33%]
test_books.py::test_word_counts[testing-3] PASSED    [ 44%]
test_books.py::test_word_counts[Prince-1498] PASSED  [ 55%]
test_books.py::test_word_counts[internet-0] PASSED   [ 66%]
test_books.py::test_word_counts[Russia-71] PASSED    [ 77%]
test_books.py::test_word_counts[Pierre-1260] PASSED  [ 88%]
test_books.py::test_word_counts[None-566311] PASSED  [100%]

==================== 9 passed in 24.42s ====================


Perhaps surprisingly, it is still taking very long time!

By default a fixture will run once for every test function that uses it. In our case we only need it to run once for all the tests in the test session so we can pass in the `scope` parameter to `pytest.fixture` and set it to `session`:

In [12]:
%%writefile test_books.py

import urllib.request

import pytest

from books import word_count

@pytest.fixture(scope="session")
def long_book():
    url = "https://www.gutenberg.org/files/2600/2600-0.txt"
    book_text = urllib.request.urlopen(url).read().decode('utf-8')
    return book_text

@pytest.mark.parametrize('word, count',  [
    ('hat', 33),
    ('freedom', 71),
    ('electricity', 1),
    ('testing', 3),
    ('Prince', 1498),
    ('internet', 0),
    ('Russia', 71),
    ('Pierre', 1260),
    (None, 566311),
])
def test_word_counts(long_book, word, count):
    assert word_count(long_book, word) == count

Overwriting test_books.py


In [13]:
!COLUMNS=60 venv/bin/pytest -v test_books.py

=================== test session starts ====================
platform linux -- Python 3.7.3, pytest-5.2.2, py-1.8.0, pluggy-0.13.0 -- /home/matt/projects/software_engineering_best_practices/venv/bin/python3
cachedir: .pytest_cache
rootdir: /home/matt/projects/software_engineering_best_practices
plugins: nbval-0.9.3
collected 9 items                                          

test_books.py::test_word_counts[hat-33] PASSED       [ 11%]
test_books.py::test_word_counts[freedom-71] PASSED   [ 22%]
test_books.py::test_word_counts[electricity-1] PASSED [ 33%]
test_books.py::test_word_counts[testing-3] PASSED    [ 44%]
test_books.py::test_word_counts[Prince-1498] PASSED  [ 55%]
test_books.py::test_word_counts[internet-0] PASSED   [ 66%]
test_books.py::test_word_counts[Russia-71] PASSED    [ 77%]
test_books.py::test_word_counts[Pierre-1260] PASSED  [ 88%]
test_books.py::test_word_counts[None-566311] PASSED  [100%]

==================== 9 passed in 3.45s =====================


Now it only takes about as long as a single test did since the slow part is only being done once.

### Exercise

Add some more parameters to the test and check that it doesn't take any longer to run